# sommelier.ai
#### Practical Machine Learning Workshop

### Agenda:
- Data Exploration with pandas
- Modeling with scikit-learn

### Documentation
- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) docs
- [Modern Pandas](https://tomaugspurger.github.io/modern-1-intro.html) blog series
- [scikit-learn](http://scikit-learn.org/stable/index.html)
- [matplotlib](https://matplotlib.org/api/api_overview.html)

## Data Exploration

In [ ]:
# These 'magics' alter the behavior of the Jupyter notebook
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

In [ ]:
# This cell is all for styling graphs

import matplotlib.pyplot as plt
import seaborn as sns

from workshop import boxplot_sorted

sns.set(style="darkgrid")
plt.style.use("fivethirtyeight")

# https://matplotlib.org/tutorials/introductory/customizing.html
plt.rc("figure", figsize=(11, 8))

FONT_SIZE = 16

plt.rc("font", size=FONT_SIZE)          # controls default text sizes
plt.rc("axes", titlesize=FONT_SIZE)     # fontsize of the axes title
plt.rc("axes", labelsize=FONT_SIZE)     # fontsize of the x and y labels
plt.rc("xtick", labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc("ytick", labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc("legend", fontsize=FONT_SIZE)    # legend fontsize
plt.rc("figure", titlesize=FONT_SIZE)   # fontsize of the figure title

In [ ]:
wine_reviews: Path = Path.cwd() / "data" / "winemag-data.zip"

df: pd.DataFrame = pd.read_csv(wine_reviews)

# Display the first few elements
df.head()

In [ ]:
len(df)

## Indexing

- Use `.loc` for label-based indexing
- Use `.iloc` for positional indexing

You can omit these, but the [Zen Of Python](https://www.python.org/dev/peps/pep-0020/) say __explicit is better than implicit__ and there is a performance penalty from pandas having to infer which indexing method to use.

In [ ]:
df.loc[(df.province == "Washington") & (df.points > 98)]

In [ ]:
df.iloc[15:18]

In [ ]:
df.iloc[15:18]

In [ ]:
# Selecting columns
df[["country", "winery"]]

## How are wines scored?

In [ ]:
df.points.describe()

In [ ]:
df.points.plot.hist(title="Points")

### How long are descriptions?

This examples shows creating a new column and using the "str" extension for [working with text data](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html).

In [ ]:
df["word_count"] = df["description"].str.split().apply(len)
df

In [ ]:
df["word_count"].plot.hist(title="Word Count");

In [ ]:
# Gets the longest description by word count.
df.iloc[df.word_count.idxmax].description

In [ ]:
df.iloc[df.word_count.idxmin].description

## What are the most common words used to describe wines?

In [ ]:
%%time

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
document_term_matrix = vectorizer.fit_transform(df["description"])
term_df = pd.DataFrame.sparse.from_spmatrix(document_term_matrix, columns=vectorizer.get_feature_names())
term_df

In [ ]:
count_df.sum().sort_values(ascending=False).head(80)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
with pd.option_context("display.max_rows", 100):
    # display is what is used to show an item when it is last element in a cell.
    display(count_df.sum().sort_values(ascending=False).head(80))

In [ ]:
# Another command option is setting the max_colwidth to see the complete contents.
with pd.option_context("display.max_colwidth", -1):
    display(df["description"].tail(10))

## Aside: `inplace=True`

Pandas maintainers say don't use it. They are [trying to deprecate it](https://github.com/pandas-dev/pandas/issues/16529).

![caption](images/inplace.png)

## How many tasters are there?

In [ ]:
len(df.taster_name.unique())

In [ ]:
# Need to be careful of dropna across several different methods, like value_counts.
df.taster_name.value_counts()

In [ ]:
# Need to be careful of dropna across several different methods, like value_counts.
df.taster_name.value_counts(dropna=False)

In [ ]:
# Setting a value with a row and column indexer.
df.loc[df.taster_name.isna(), "taster_name"] = "Unknown"

In [ ]:
df.taster_name.value_counts()

## What are the top 20 wineries by number of wines? How do their points compare?

In [ ]:
df.winery.isna().any()

In [ ]:
# Get an index made up from the twenty most productive wineries (discard the value counts).
top20 = df.winery.value_counts()[:20].index
top20

In [ ]:
# Method chaining is considered "good style" in pandas.
(df.loc[df.winery.isin(top20)]
   .groupby("winery")
   .points
   .quantile(0.95)
   .sort_values(ascending=False))

In [ ]:
boxplot_sorted(df[df.winery.isin(top20)], by="winery", column="points");

## Are some tasters pickier than others?

In [ ]:
boxplot_sorted(df, by="taster_name", column="points");

In [ ]:
def get_favorite_wines(name: str) -> pd.Series:
    return (df.loc[df.taster_name == name]
              .groupby("variety")
              .points
              .quantile(0.95)
              .sort_values(ascending=False)
              .head())

get_favorite_wines("Virginie Boone")

In [ ]:
get_favorite_wines("Alexander Peartree")

## Data Challenges
- what are the worst wines in the US?
- what is the most produced variety?
- what is hightest rated variety?
- what are the top 10 best value wines?

In [ ]:
df.points.isna().any()

## What are the ten best value wines?

In [ ]:
priced_df = df.dropna(subset=["price"])
priced_df["value"] = priced_df.points / priced_df.price
priced_df.sort_values(["value"], ascending=False)[["title", "points", "price", "value"]].head(15)

## Extract Year from Title

In [ ]:
def compute_year(df: pd.DataFrame) -> pd.DataFrame:
    # It is polite not to alter the input DataFrame
    result: pd.DataFrame = df.copy()

    year: pd.Series = df.title.str.extract(r"\b((?:19|20)\d{2})\b")

    # Get the median year
    median: int = round(year.dropna().astype(int).median())
    
    # Replace NaNs with the median
    result["year"] = year.replace(pd.NA, median).astype(int)

    return result

In [ ]:
df = df.pipe(compute_year)
df

In [ ]:
# Varieties with the most variance in points
(df.groupby("variety")
   .points
   .var()
   .dropna()
   .sort_values()
   .tail(15)
   .plot
   .barh());

# A very simple sentiment analysis model

In [ ]:
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from workshop import show_most_informative_features

def evaluate(model, X, y):
    predictions = model.predict(X)
    score = metrics.accuracy_score(y, predictions)
    print('\nAccuracy: %0.3f' % score)

    print(metrics.classification_report(y, predictions))

In [ ]:
df["is_good"] = df.points > 90

train_df, test_df, train_labels, test_labels = train_test_split(
    df.drop(columns=["is_good", "price", "points"]), 
    df.is_good,
    random_state=3)

In [ ]:
%%time

count_model = make_pipeline(CountVectorizer(), MultinomialNB())

count_model.fit(train_df.description, train_labels)

evaluate(count_model, test_df.description, test_labels)

In [ ]:
%%time

tf_idf_model = make_pipeline(
            make_column_transformer(
                (TfidfVectorizer(ngram_range=(1,3), max_df=0.98), "description")),
            SGDClassifier(n_jobs=-1, loss="modified_huber"))

tf_idf_model.fit(train_df, train_labels)

evaluate(tf_idf_model, test_df, test_labels)

In [ ]:
show_most_informative_features(tf_idf_model)

In [ ]:
%%time

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"))

model = make_pipeline(
            make_column_transformer(
                (TfidfVectorizer(ngram_range=(1,3), max_df=0.98), "description"),
                (categorical_transformer, ["country", "winery"]),
                (make_pipeline(
                    SimpleImputer(strategy="median"),
                    StandardScaler()), ["year"])),
            SGDClassifier(n_jobs=-1, loss="modified_huber"))

model.fit(train_df, train_labels)

predicted = model.predict(test_df)

evaluate(model, test_df, test_labels)

In [ ]:
show_most_informative_features(model)

In [ ]:
def get_failures(model, X, y, cv=3):
    predicted = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)

    print("Confusion matrix (actual x prediction):")
    print(metrics.confusion_matrix(y, predicted))

    fn = X[(y == True) & (predicted == False)]
    fp = X[(y == False) & (predicted == True)]

    return fn, fp

In [ ]:
fn, tp = get_failures(model, train_df, train_labels)